In [1]:
import json
import os
import logging
import sys

from autosted.callback_buildingblocks.regular_position_generators import SpiralOffsetGenerator
from autosted.callback_buildingblocks.coordinate_value_wrappers import StageOffsetsSettingsGenerator, ScanOffsetsSettingsGenerator
from autosted.callback_buildingblocks.value_wrappers import SimpleManualOffset
from autosted.callback_buildingblocks.parameter_filtering import LocationRemover, LocationKeeper
from autosted.callback_buildingblocks.static_settings import FOVSettingsGenerator, DifferentFirstFOVSettingsGenerator, ScanModeSettingsGenerator
from autosted.callback_buildingblocks.data_selection import NewestDataSelector, NewestSettingsSelector
from autosted.callback_buildingblocks.static_settings import JSONSettingsLoader
from autosted.detection import SimpleFocusPlaneDetector
from autosted.detection.legacy import LegacySpotPairFinder, SimpleSingleChannelSpotDetector
from autosted.stoppingcriteria import TimedStoppingCriterion
from autosted.taskgeneration import AcquisitionTaskGenerator
from autosted.pipeline import AcquisitionPipeline
from autosted.imspector import get_current_stage_coords

# configure logging
logging.basicConfig(stream=sys.stdout, level=logging.INFO)

In [2]:
def run_pipeline(save_path, hours_to_image, ov_json, det_jsons, start_coords, sigma,
                 thresholds, ov_moves, ov_fovs=None, ov_psz=None, ov_fovs_first=None, 
                 det_fovs=None, det_pszs=None, ov_json_imspector=None, det_jsons_imspector=None,
                 onecolor=False, sample_name=None, focus_channel=0, manual_focus_offset=[0,0,0],
                 ov_mode=None, det_modes=None, between_channel_max_distance=5, channels_singlecolor_detection=0):

    # init pipeline with hierarchy levels 'field' (overview) and 'sted' (detail)
    pipeline = AcquisitionPipeline(
        name=('spot-pair-pipeline' if sample_name is None else sample_name),
        data_save_path=save_path,
        hierarchy_levels=('field', 'sted'),
        save_combined_hdf5=True)

    # overview task generator (for 'field'-level images): move in spiral and hold focus
    next_overview_generator = AcquisitionTaskGenerator('field',
        # 1. load settings from JSON, remove any location parameters
        LocationRemover(JSONSettingsLoader(ov_json, ov_json_imspector, as_measurements=False)),
        # 2. set stage position based on positions from spiral generator
        StageOffsetsSettingsGenerator(SpiralOffsetGenerator(ov_moves, start_coords)),
        # 3. maually switch scan mode (by default, do xyz)
        ScanModeSettingsGenerator(['xyz' if ov_mode is None else ov_mode]),
        # 4. set FOV size, first image can have a different FOV (e.g. for easier focus)
        DifferentFirstFOVSettingsGenerator(ov_fovs, ov_psz, ov_fovs_first),
        # 5. set z stage position based on simple autofocus (brightest plane) in previous overview, optionally add offset
        StageOffsetsSettingsGenerator(
            SimpleManualOffset(
                SimpleFocusPlaneDetector(channel=focus_channel),
                offset=manual_focus_offset)
            )
        )

    # spot detector callback using the newest overview
    # can be pair detector or single channel detector
    if not onecolor:
        detector = LegacySpotPairFinder(NewestDataSelector(pipeline, 'field'),
            sigma, thresholds, between_channel_max_distance=between_channel_max_distance,
            in_channel_min_distance=3, plot_detections=True)
    else:
        detector = SimpleSingleChannelSpotDetector(NewestDataSelector(pipeline, 'field'),
            sigma, thresholds, channel=channels_singlecolor_detection,
            refine_detections=False, plot_detections=True)

    # detail task generation: settings from last overview + new FOV + detection
    # optionally repeat measurement to check reproducibility
    detail_generator = AcquisitionTaskGenerator('sted',
        # 1. take position of the overview image, ignore other parameters
        LocationKeeper(NewestSettingsSelector(pipeline, 'field')),
        # 2. take all other parameters from saved settings
        LocationRemover(JSONSettingsLoader(det_jsons, det_jsons_imspector, False)),
        # 3. manually switch scan mode
        ScanModeSettingsGenerator(det_modes if not det_modes is None else ['xyz'] * len(det_jsons), False),
        # 4. manually switch FOV size / pixel size
        FOVSettingsGenerator(det_fovs, det_pszs),
        # 5. set scan offset based on detections
        ScanOffsetsSettingsGenerator(detector)
    )

    # add next overview and detail generation callbacks to pipeline
    pipeline.add_callback(next_overview_generator, 'field')
    pipeline.add_callback(detail_generator, 'field')
    
    # add stopping criterion
    pipeline.add_stopping_condition(TimedStoppingCriterion(hours_to_image * 60 * 60))

    # run pipeline, use overview generator to set initial task
    pipeline.run(next_overview_generator)

In [5]:
# init acquisition list
# re-run to clear
acquisitions = []

In [ ]:
params = {}

# where to save
params['save_path'] = 'D:/AUTOMATION/TEST/2color/raw'

# how long to image
params['hours_to_image'] = 1

### MEASUREMENT PARAMETER FILES
# paths of the parameters files
# we can use multiple for the STED/detail measurement (e.g. to do both a 2d and 3d STED acq.)
params['ov_json'] = 'examples/config_json/test2color_60x_overview.json'
params['det_jsons'] = ['examples/config_json/test2color_60x_detail.json']

### IMSPECTOR / HARDWARE PARAMETERS, optional
# paths to imspector setting files, set to None if you do not want to change settings (e.g. SLM parameters)
params['ov_json_imspector'] = None
params['det_jsons_imspector'] = None
# NOTE: needs to be of the same format as ov_json and det_jsons (one overview, one or more detail)
# uncomment lines below for example
#params['ov_json_imspector'] = 'config_json/imspector_hardware_settings.json'
#params['det_jsons_imspector'] = ['config_json/imspector_hardware_settings2.json']


### DETECTOR SETTINGS
# LoG spot detection parameters (~expected size, thresholds for each channel)
params['sigma'] = 2
params['thresholds'] = [1, 3]

# whether to detect only in individual channels (and not look for pairs)
params['onecolor'] = True
# NOTE: by passing a list of channels in addition to onecolor=True,
# spots will be detected in each of the channels independently
# and detail images will be acquired for spots in any channel
# Alternative: comment out to detect in just the first channel
params['channels_singlecolor_detection'] = [1]

# for pair / twocolor detection: maximum distance between 
params['between_channel_max_distance'] = 7


### FOV and pixel size of overviews (optional)
# NOTE: we can set a bigger z stack for first stack
# NOTE: all sizes are in meters!

params['ov_fovs'] = [[0.5e-05, 5e-05, 5e-5]]
params['ov_psz'] = [[4e-7, 2.5e-7, 2.5e-7]]
params['ov_fovs_first'] = [[0.5e-5, 5e-05, 5e-05]]
params['ov_moves'] = [5e-5, 5e-5] # how much to move in overview spiral
# (NOTE: we make it larger than FOV to avoid small overlaps)
#params['ov_moves'] = [6e-5, 6e-5] # how much to move in overview spiral

# FOVS / PSZ for details
#params['det_fovs'] = [[4e-06, 4e-06, 1.75e-6]] # STED FOV
#params['det_pszs'] = [[4.5e-8, 4.5e-8, 6e-8]] # STED Pixelsize

# autofocus options
params['focus_channel'] = 0
params['manual_focus_offset'] = [0,0,0]


### SCAN MODES (Optional)
# which scan mode (xy, xyz, ...) to use for overviews and details
# This may be None, in which case we simply use mode set in file
params['ov_mode'] = None
# e.g.
#params['ov_mode'] = 'xyz'
#params['det_modes'] = ['xyz']
# e.g.
#params['det_modes'] = ['xyz', 'xy'] 
# NOTE: this needs to be a list of the same size as settings for details


### Things that are set automatically

# ensure we use slashes and not backslashes
params['save_path'] = params['save_path'].replace(os.sep, '/')
# sample name, will be set automatically
params['sample_name'] = params['save_path'].strip('/').rsplit('/')[-1]
# start at current coordinates, do not change!
params['start_coords'] = get_current_stage_coords()

# add to queue
acquisitions.append(params)

# print the currently queued acquisitions
print("""
Queued Acquisitions:
====================
""")
for ac in acquisitions:
    print(json.dumps(ac, indent=1))

In [ ]:
# go through queued acquisitions (in reverse) and run them
do_reversed = True
for ac in reversed(acquisitions) if do_reversed else acquisitions:
    run_pipeline(**ac)

# Reset queued acquisitions
# NOTE: if you cancelled a run, this might not be executed,
#     make sure to clear old acquisitions manually (step 1)
acquisitions = []